In [1]:
import hdbscan
import pickle
import pandas as pd
from typing import Tuple
import pathlib

In [2]:
def import_clusters(
    path_to_tsv: pathlib.Path, path_to_picke: pathlib.Path
) -> Tuple[pd.DataFrame, hdbscan.HDBSCAN]:
    df = pd.read_table(path_to_tsv)
    with open(path_to_picke, "rb") as f:
        return df, pickle.load(f)

In [3]:
df, clusters = import_clusters(
    "../data/output/comparative_analysis/annotated_domains/50000/clusters/clique.clusters.tsv.gz",
    "../data/output/comparative_analysis/annotated_domains/50000/clusters/clique.clusterer.pickle.xz",
)

In [4]:
df1 = df.groupby(["label", "cluster"]).size().to_frame()
df1.columns = ["size"]
df1

size
label     cluster      
MCF10A    -1       1377
           0         57
           1         41
           2         36
           3         35
           4         36
           5        443
           6         48
           7        539
           8        314
MCF10AT1  -1        819
           0         64
           1         15
           2         64
           3         65
           4         56
           5         63
           6         43
           7        321
           8        398
MCF10CA1a -1        940
           0         14
           1         65
           2         67
           3          4
           4         29
           5        235
           6        106
           7        384
           8        420